<a href="https://colab.research.google.com/github/sebasanjose/AI-Notebooks/blob/main/06_errors_01_add_error_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile 06-errors-01-add-error-handling.cu
#include <stdio.h>

void init(int *a, int N)
{
  int i;
  for (i = 0; i < N; ++i)
  {
    a[i] = i;
  }
}

__global__
void doubleElements(int *a, int N)
{

  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = gridDim.x * blockDim.x;

  for (int i = idx; i < N + stride; i += stride)
  {
    a[i] *= 2;
  }
}

bool checkElementsAreDoubled(int *a, int N)
{
  int i;
  for (i = 0; i < N; ++i)
  {
    if (a[i] != i*2) return false;
  }
  return true;
}

int main()
{
  /*
   * Add error handling to this source code to learn what errors
   * exist, and then correct them. Googling error messages may be
   * of service if actions for resolving them are not clear to you.
   */

  int N = 10000;
  int *a;

  size_t size = N * sizeof(int);
  cudaMallocManaged(&a, size);

  init(a, N);

  size_t threads_per_block = 2048;
  //size_t threads_per_block = 1024;
  size_t number_of_blocks = 32;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);
  cudaError_t sync_err, async_err;
  sync_err = cudaGetLastError(); // `cudaGetLastError` will return the error from above.
  if (sync_err != cudaSuccess)
  {
    printf("Sync Error: %s\n", cudaGetErrorString(sync_err));
  }

  async_err = cudaDeviceSynchronize();
  if (async_err != cudaSuccess)
  {
    printf("Async Error: %s\n", cudaGetErrorString(async_err));
  }

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  cudaFree(a);
}


Writing 06-errors-01-add-error-handling.cu


In [2]:
!nvcc 06-errors-01-add-error-handling.cu -o 06-errors-01-add-error-handling -run

Sync Error: invalid configuration argument
All elements were doubled? FALSE
